![Texto Alternativo](banco_de_dados_projeto.png)

Dados de vendas de 2016~2018 da Olist ecommerce

https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce#olist_products_dataset.csv

https://www.kaggle.com/datasets/thiagobodruk/brazil-geojson

https://github.com/filipegmedeiros/brazilian_choropleth_map/tree/main/geojson

In [1]:
import pandas as pd
import plotly.express as px
import json
import plotly.io as pio
pio.renderers.default = 'jupyterlab'

In [2]:
# Carga inicial dos dados
df_order_payments = pd.read_csv("data/olist_order_payments_dataset.csv")
df_order_dataset = pd.read_csv("data/olist_orders_dataset.csv")
df_order_customer = pd.read_csv("data/olist_customers_dataset.csv")
df_order_itens = pd.read_csv("data/olist_order_items_dataset.csv")
df_order_products = pd.read_csv("data/olist_products_dataset.csv")
df_order_sellers = pd.read_csv("data/olist_sellers_dataset.csv")
df_order_geolocation = pd.read_csv("data/olist_geolocation_dataset.csv")

# Geojson
geojson = json.load(open("geojson/brasil_estados.json"))

In [13]:
# Merge dos dados
df_order_geolocation.rename(columns={'geolocation_zip_code_prefix' : 'zip_code_prefix'}, inplace=True)
df_unido = pd.merge(df_order_payments, df_order_dataset, how="inner", on="order_id")
df_unido = pd.merge(df_unido, df_order_itens, how="inner", on="order_id")
df_unido = pd.merge(df_unido, df_order_customer, how="inner", on="customer_id")
df_unido = pd.merge(df_unido, df_order_products, how="inner", on="product_id")
df_unido = pd.merge(df_unido, df_order_sellers, how="inner", on="seller_id")
df_unido.rename(columns={'customer_zip_code_prefix' : 'zip_code_prefix'}, inplace=True)

In [15]:
df_unido_amostra = df_unido.sample(frac=0.2, random_state=42)
df_unido_amostra.to_parquet("data/amostra.parquet")

In [ ]:
tipo_pagamento = df_unido_amostra.groupby(by='payment_type').size().reset_index().sort_values(by=0,
                                                                                              ascending=False)

In [ ]:
compra_por_estado = df_unido_amostra.groupby(by='customer_state').size().sort_values(ascending=False).reset_index()

In [ ]:
# location tem que fazer correlação com a coluna ID do geojson
fig = px.choropleth(compra_por_estado, geojson=geojson, locations='customer_state', color=0, scope="south america")
fig.update_geos(fitbounds = "locations", visible = False)
fig.update_layout(width=800, height=800,dragmode=False,paper_bgcolor="white")
fig.show()

In [ ]:
compra_por_cidade = df_unido_amostra.groupby(by='customer_city').size().sort_values(ascending=False).reset_index()